# Day 4 - Advent of Code 2020

## Part 1

In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:,.4f}'.format

import re

In [2]:
with open('day_04.txt') as f:
    data = f.read()
    data_list = data.split('\n')

data_list[0:8]

['eyr:2029 pid:157374862',
 'byr:1991 ecl:amb hcl:#a97842 hgt:178cm',
 '',
 'byr:1962 pid:547578491 eyr:2028 ecl:hzl hgt:65in iyr:2013 hcl:#623a2f',
 '',
 'hgt:71in eyr:2037',
 'ecl:#8e276e hcl:z iyr:2019',
 'byr:2022 pid:157cm']

In [3]:
l = []
for i in data_list:
    if len(i) == 0:
        l.append(1)
len(l) + 1

292

In [4]:
pass_list = ['']

for i in data_list:
    if len(i) > 1:
        pass_list[-1] += i + ' '
    else:
        pass_list += ['']

pass_list[0:3]

['eyr:2029 pid:157374862 byr:1991 ecl:amb hcl:#a97842 hgt:178cm ',
 'byr:1962 pid:547578491 eyr:2028 ecl:hzl hgt:65in iyr:2013 hcl:#623a2f ',
 'hgt:71in eyr:2037 ecl:#8e276e hcl:z iyr:2019 byr:2022 pid:157cm ']

In [5]:
passports = []

for i, p in enumerate(pass_list, start=1):
    cred_dict = {
        'pass_id': i,
        'byr': np.NaN, 
        'iyr': np.NaN, 
        'eyr': np.NaN, 
        'hgt': np.NaN, 
        'hcl': np.NaN, 
        'ecl': np.NaN, 
        'pid': np.NaN, 
        'cid': np.NaN
    }

    for cred in cred_dict.keys():
        if re.search(fr'(?<={cred}:)\S+', p):
            cred_dict[cred] = re.search(fr'(?<={cred}:)\S+', p)[0]
    
    passports.append(cred_dict)

df = pd.DataFrame(passports).astype('string')
df[0:5]

,pass_id,byr,iyr,eyr,hgt,hcl,ecl,pid,cid
0,1,1991,<NA>,2029,178cm,#a97842,amb,157374862,<NA>
1,2,1962,2013,2028,65in,#623a2f,hzl,547578491,<NA>
2,3,2022,2019,2037,71in,z,#8e276e,157cm,<NA>
3,4,1980,2011,2028,175,#733820,hzl,87786389,<NA>
4,5,2030,1991,1958,163in,z,grn,8261494508,136


In [6]:
pd.DataFrame(df.isna().sum(), columns=['nulls']).T

,pass_id,byr,iyr,eyr,hgt,hcl,ecl,pid,cid
nulls,0,7,11,6,7,7,6,7,158


In [7]:
df[~df.loc[:, df.columns[1:-1]].isna().any(axis=1)].shape[0]

245

## Part 2

In [8]:
df_valid = df[~df.loc[:, df.columns[1:-1]].isna().any(axis=1)].copy()

In [9]:
def pass_validity(row):
    byr_rule = 0 if row['byr'].isdigit() and int(row['byr']) in range(1920, 2003) else 1
    iyr_rule = 0 if row['iyr'].isdigit() and int(row['iyr']) in range(2010, 2021) else 1
    eyr_rule = 0 if row['eyr'].isdigit() and int(row['eyr']) in range(2020, 2031) else 1
    hgt_rule = 0 if row['hgt'][-2:] in ['cm', 'in'] and row['hgt'][0:-2].isdigit() and (((row['hgt'][-2:] == 'cm') and (int(row['hgt'][0:-2]) in range(150, 194))) or ((row['hgt'][-2:] == 'in') and (int(row['hgt'][0:-2]) in range(59, 77)))) else 1
    hcl_rule = 0 if re.search(r'#[0-9a-f]{6}', row['hcl']) else 1
    ecl_rule = 0 if row['ecl'] in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'] else 1
    pid_rule = 0 if row['pid'].isdigit() and len(row['pid']) == 9 else 1
    
    return 0 if np.sum([byr_rule, iyr_rule, eyr_rule, hgt_rule, hcl_rule, ecl_rule, pid_rule]) > 0 else 1

In [10]:
df_valid['valid'] = df_valid.apply(pass_validity, axis=1)
df_valid[df_valid['valid'] == 1].shape[0]

133